In [17]:
import numpy as np
import pandas as pd
import re
import nltk
import string
import couchdb
# from couchdb_settings import *
import geojson
from geojson import Point, Feature, FeatureCollection, dump
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from textblob import TextBlob
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/weimin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/weimin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
# couchdb_settings
address = '172.26.134.35:5984'
username = 'grp5admin'
password = 'password'
tweets = 'raw_tweets'
user = 'user_list'

In [19]:
def db_connect(dbname):
    """
    connect to CouchDB server to create databases
    params: name of the database to be created
    return: created database
    """

    couchserver = couchdb.Server('http://' + username + ':' + password + '@' + address)
    try:
      db = couchserver[dbname]
    except:
      db = couchserver.create(dbname)

    return db

In [20]:
def fetch_DB(dbname):
    """
    connect to CouchDB
    params: name of the database to connect to
    return: the db to establish connection with
    return type: database
    """
    
    db = couchdb.Database('http://' + address + '/' + dbname)
    db.resource.credentials = (username, password)

    return db

In [21]:
# connect to raw_tweets db
tweet_db = fetch_DB(tweets)

In [22]:
def delete_docs(topic, save_db):
    """
    delete existing data in DB (to replace previous data analyses results)
    params: the topic to look at (one of housing, cost or transportation);
            the database where the particular topic results were saved in
    """
    
    docs = []    
    for row in save_db.view(topic + '/all', include_docs=True):
        doc = row['doc']
        if int(doc['year']) >= 2018:
            doc['_deleted']=True
            docs.append(doc)
        save_db.update(docs)

In [23]:
def now_trending(db, N):
    """
    extract top N mostly used hasgtags in tweets from past 14 days
    params: raw_tweets database;
            number of hashtags to extract
    return: top N hashtags extracted from tweets made within last 14 days; 
            all hashtags in lowercases
    return type: dict - {hashtag:count}
    frontend: wordcloud
    """
    
    hashtags = {}
    
    for item in db.view('hashtags/trending', group = True, group_level = 1):
        if item.key.lower() not in hashtags.keys():
            hashtags[item.key.lower()] = item.value
        else:
            hashtags[item.key.lower()] += item.value
        
    hashtags = {k: v for k, v in sorted(hashtags.items(), key=lambda item: item[1])[-N:]}

    return hashtags

# run now_trending
now_trending(tweet_db, 10)

{'tiktok': 96,
 'alboforpm': 107,
 'gopies': 114,
 'armukrainenow': 124,
 'victraffic': 135,
 'slavaukraini': 147,
 'melbourne': 165,
 'ausvotes': 190,
 'auspol': 495,
 'امپورٹڈ_حکومت_نامنظور': 918}

In [24]:
db = fetch_DB('langcode')

In [25]:
def read_langCode(db):

    langCode = {}
    for item in db.view('lang/Code'):
        langCode[item.key] = item.value
        
    return langCode

In [13]:
langCode = read_langCode(db)

In [15]:
langCode

{'af': 'Afrikaans',
 'am': 'Amharic',
 'ar': 'Arabic',
 'arn': 'Mapudungun',
 'as': 'Assamese',
 'az': 'Azeri',
 'ba': 'Bashkir',
 'be': 'Belarusian',
 'bg': 'Bulgarian',
 'bn': 'Bengali',
 'bo': 'Tibetan',
 'br': 'Breton',
 'bs': 'Bosnian',
 'ca': 'Catalan',
 'co': 'Corsican',
 'cs': 'Czech',
 'cy': 'Welsh',
 'da': 'Danish',
 'de': 'German',
 'dsb': 'Lower-Sorbian',
 'el': 'Greek',
 'en': 'English',
 'es': 'Spanish',
 'et': 'Estonian',
 'eu': 'Basque',
 'fa': 'Persian',
 'fi': 'Finnish',
 'fil': 'Filipino',
 'fo': 'Faroese',
 'fr': 'French',
 'fy': 'Frisian',
 'ga': 'Irish',
 'gd': 'Scottish-Gaelic',
 'gl': 'Galician',
 'gsw': 'Alsatian',
 'gu': 'Gujarati',
 'ha': 'Hausa',
 'he': 'Hebrew',
 'hi': 'Hindi',
 'hr': 'Croatian',
 'hsb': 'Upper-Sorbian',
 'ht': 'Haitian',
 'hu': 'Hungarian',
 'hy': 'Armenian',
 'id': 'Indonesian',
 'ig': 'Igbo',
 'ii': 'Yi',
 'is': 'Icelandic',
 'it': 'Italian',
 'iu': 'Inuktitut',
 'ja': 'Japanese',
 'ka': 'Georgian',
 'kk': 'Kazakh',
 'kl': 'Greenlandic',

In [16]:
def top_n_lang_count(db, langCode_db, N):
    """
    extract top N languages other than English in which tweets were made
    params: raw_tweets database;
            path to langCode.json file;
            number of languages to extract
    return: top N most tweeted languages other than English
    return type: dict - {language code: count}
    frontend: bar chart/pie chart (colour matching for most tweeted languages/counrty of birth/language spoken at home)
    """
    languages = {}
    
    for item in db.view('lang/lang-count', group = True, group_level = 1):
        if item.key != 'en':
            if item.key == 'in':
                languages['id'] = item.value
            else:
                languages[item.key] = item.value
            
        languages = {k:v for k, v in sorted(languages.items(), key=lambda item: item[1])[::-1][:N]}
    
    langCode = read_langCode(langCode_db)
    
    languages = {v2: v1 for k1, v1 in languages.items() for k2, v2 in langCode.items() if k1 == k2}
            
    return languages

# run top n languages
top_n_lang_count(tweet_db, db, 10)

{'Japanese': 11960,
 'Spanish': 11814,
 'Indonesian': 8785,
 'Arabic': 6030,
 'Tagalog': 4549,
 'French': 3316,
 'Chinese': 3111,
 'Portuguese': 2853,
 'Thai': 2763,
 'Turkish': 2133}

In [26]:
birthdb = fetch_DB('birthcountry')

In [27]:
def top_n_birth_country(db, N):

    birth = {}
    for item in db.view('birth/country'):
        birth[item.key] = item.value
        
    birth = {k: v for k, v in sorted(birth.items(), key=lambda item: item[1])[-N:]}
    
    return birth

top_n_birth_country(birthdb, 10)

{'Germany': [17511, 0.38707478361743725],
 'Pakistan': [19127, 0.42279592177778097],
 'Philippines': [43642, 0.9646917769763117],
 'Greece': [43881, 0.9699747918403725],
 'Malaysia': [45852, 1.0135430859703463],
 'Sri Lanka': [52658, 1.1639874339402094],
 'Italy': [61521, 1.3599010772045204],
 'Vietnam': [78036, 1.7249596147775874],
 'India': [160058, 3.5380284230620616],
 'China': [174418, 3.85545140819977]}

In [28]:
homelang = fetch_DB('homelang')

In [29]:
def top_n_lang_spoken_at_home(db, N):

    spoken = {}
    for item in db.view('home/lang'):
        spoken[item.key] = item.value
        
    spoken = {k: v for k, v in sorted(spoken.items(), key=lambda item: item[1])[-N:]}
    
    return spoken

top_n_lang_spoken_at_home(homelang, 10)

{'Spanish': [27632, 2.376169510181618, 0.6908029359124777],
 'Macedonian': [29378, 2.526313979086406, 0.734453121425766],
 'Punjabi': [29517, 2.5382670610897082, 0.7379281361945789],
 'Turkish': [31257, 2.6878955696202533, 0.7814283210703645],
 'Hindi': [31607, 2.717993258117777, 0.7901783582580226],
 'Arabic': [65454, 5.628611722619703, 1.6363569545170566],
 'Vietnamese': [85122, 7.319929829389103, 2.1280590442509384],
 'Italian': [112665, 9.68844592735278, 2.8166369707071257],
 'Greek': [113424, 9.753714914694552, 2.835612051351218],
 'Chinese': [189854, 16.326190148596588, 4.746370172073231]}

In [30]:
def topic_switch(topic):
    """
    params: topic of selection
    return: paths to the views relating to the selected topic;
            connect to db where top related results were saved
    """

    count_view = 'text/' + topic + '-count'
    topic_view = 'text/' + topic

    return count_view, topic_view

In [31]:
def topic_trend(db, topic):
    """
    extract the number and percentage of tweets on the selected topic made each year
    params: raw_tweets database;
            the topic of selection
    return:  
    return type: dict - {year : number of tweets on the selected topic made in that year}
                 dict - {year : total number of tweets made in that year}
                 dict - {year : percentage of tweets on selected topic over total number of tweets made in that year}
    frontend: Dual axes, line and column (combine with topic sentiment as the line)
    """

    count_view, _ = topic_switch(topic)
    
    year_topic = {}
    year_total = {}
    percent = {}

    for item in db.view(count_view, group = True, group_level = 1):
        year_topic[item.key] = item.value

    for item in db.view('time/by-year-count', group = True, group_level = 1):
        year_total[item.key] = item.value

    for key in year_topic.keys():
        percent[key] = year_topic[key]/year_total[key] * 100
            
    return year_topic, year_total, percent

In [32]:
topics = ['housing', 'cost', 'transportation']

for topic in topics:
    trend = topic_trend(tweet_db, topic)
    print(trend)

({2014: 165, 2015: 154, 2016: 7, 2017: 365, 2018: 8, 2019: 18, 2020: 50, 2021: 168, 2022: 187}, {2014: 211755, 2015: 198258, 2016: 10801, 2017: 107505, 2018: 9586, 2019: 15241, 2020: 35793, 2021: 86473, 2022: 99981}, {2014: 0.07792023801090883, 2015: 0.07767656286253266, 2016: 0.06480881399870382, 2017: 0.33951909213524956, 2018: 0.08345503859795535, 2019: 0.11810248671347025, 2020: 0.13969211857067024, 2021: 0.19428029558359255, 2022: 0.18703553675198287})
({2014: 4, 2015: 5, 2017: 4, 2019: 1, 2021: 4, 2022: 37}, {2014: 211755, 2015: 198258, 2016: 10801, 2017: 107505, 2018: 9586, 2019: 15241, 2020: 35793, 2021: 86473, 2022: 99981}, {2014: 0.0018889754669311233, 2015: 0.0025219663267056058, 2017: 0.0037207571740849265, 2019: 0.006561249261859458, 2021: 0.00462572132341887, 2022: 0.03700703133595383})
({2014: 792, 2015: 1044, 2016: 93, 2017: 2439, 2018: 15, 2019: 59, 2020: 57, 2021: 119, 2022: 567}, {2014: 211755, 2015: 198258, 2016: 10801, 2017: 107505, 2018: 9586, 2019: 15241, 2020: 3

In [33]:
def topic_wordcloud_save(query_db, topic, save_db):

    # fetch the view to access & connect to the database where the topic related results were saved
    _, topic_view = topic_switch(topic)

    # if there were results saved from previous data analyses runs, delete 
    # (because there would have been new data collected by the Twitter-Harvester so the results would have changed)
    try:
        delete_docs(topic, save_db)
    except Exception:
        pass
    
    yearly_tweets = defaultdict(list)
    for item in query_db.view(topic_view):
        if int(item.key) >= 2018:
            yearly_tweets[item.key].append(item.value)
    
    tokenizer = TweetTokenizer()
    
    for key, tweet in yearly_tweets.items():
        tweet = [' '.join(re.sub("(@[A-Za-z0-9\_]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",t).split()) for t in tweet]
        tweet = [' '.join(tweet)]
        tweet_tokens = tokenizer.tokenize(tweet[0])
        tweet_clean = []
        for word in tweet_tokens:
            if word.lower() not in stopwords.words('english') and word.lower() not in string.punctuation:
                tweet_clean.append(word.lower())
                
        yearly_tweets[key] = ' '.join(tweet_clean)

    # save data analyses results to the topic related results database for the backend to directly access through views
    for k, v in yearly_tweets.items():
        save_db.save({'year': k, 'text':v})

In [34]:
# run topic wordcloud
for topic in topics:
    save_db = db_connect(topic + '_text') 
    topic_wordcloud_save(tweet_db, topic, save_db)

In [35]:
def topic_word_cloud(topic):
    """
    extract topic related wordcloud
    params: topic related text database;
            topic of selection
    return: corpus of combined tweets on the selected topic indexed by year; 
            all words in lowercases
    return type: dict - {year : corpus as a list}
    frontend: wordcloud
    """

    yearly_tweets = {}
    for item in db.view(topic + '/text'):
        yearly_tweets[item.key] = item.value   

    return yearly_tweets

In [36]:
for topic in topics:
    db = fetch_DB(topic + '_text')
    topic_word_cloud(topic)

In [37]:
def topic_sentiment(topic):
    """
    extract topic related sentiment
    params: topic of selection
    return: sentiment towards the selected topic indexed by year
    return type: dict - {year : sentiment score}
    render: Dual axes, line and column (combine with topic trend as the columns)
    """
    
    yearly_tweets = {}
    for item in db.view(topic + '/text'):
        yearly_tweets[item.key] = item.value
    
    yearly_sentiment = {}
    for key, value in yearly_tweets.items():
        blob = TextBlob(value)
        for sentence in blob.sentences:
            sentiment = sentence.sentiment.polarity
            yearly_sentiment[key] = sentiment

    return yearly_sentiment

In [38]:
for topic in topics:
    db = fetch_DB(topic + '_text')
    print(topic_sentiment(topic))

{2014: 0.19313898143741073, 2015: 0.07747157991060426, 2016: 0.22619047619047622, 2017: 0.07160565243311717, 2018: 0.10800000000000001, 2019: 0.13429682929682932, 2020: 0.10835478680611424, 2021: 0.10943601545630192, 2022: 0.10005737447910346}
{2014: 0.21095238095238095, 2015: -0.1708333333333333, 2017: -0.02857142857142858, 2019: 0.1375, 2021: 0.02727272727272726, 2022: 0.05202380952380951}
{2014: -0.0066004984047371386, 2015: -0.013641858141858245, 2016: -0.005781702037799617, 2017: -0.032537428863271586, 2018: 0.2530917280917281, 2019: 0.05319404621352675, 2020: 0.08493145743145744, 2021: 0.06922077922077922, 2022: -0.014736664490401105}


In [39]:
# backend to access data analyses results directly from CouchDB:
# e.g topic: housing - other available topics: cost, transportation

db = fetch_DB(topic + '_text') # fetch the database where the topic related results were saved

yearly_tweets = {}
for item in db.view(topic + '/text'):
    yearly_tweets[item.key] = item.value

# results returned as a dict in the form of {year: corpus}
# fetch the corpus corresponding to the selected year, or combine all corpus into one, to render for the frontend

In [41]:
def geo_LatLong(db):
    """
    extract langitude and longitude information if tweets contain the information
    params: raw_tweets database
    return: coordinates of tweets that contain the information
    return type: geojson
    frontend: map
    """
    features = []
    for item in db.view('geo/new-view'):
        cor = item.key
        features.append(Feature(geometry=Point((cor[0], cor[1]))))

    feature_collection = FeatureCollection(features)
#     with open('myfile1.geojson', 'w') as f:
#         dump(feature_collection, f)
    return feature_collection
geo_LatLong(tweet_db)

## For Data Processing ONLY

In [ ]:
def save_langCode(langCode_path):
    """
    param: language code file path
    return: {language_code: language_name} - language code dictionary
    """

    langCode_db = db_connect('langcode')
    with open(langCode_path, 'r', encoding= 'utf-8') as f:
        for line in f:
            (val, key) = line.split()
            langCode_db.save({'code': key, 'language':val})

In [67]:
def birth_country(file_path):
    """
    extract top N non-English-speaking countries where people living in the Greater Melbourne were originally from
    params: path to census data download from AURIN - 'country_of_birth.csv';
            number of non-English-speaking countries to extract
    return: top N non-English-speaking countries' names, total population count, and percentage population
    return type: numpy arrays
    frontend: bar chart/pie chart (colour matching for most tweeted languages/counrty of birth/language spoken at home)
    """
    data = pd.read_csv(file_path)
    
    match_cols = []
    new_cols = []
    col_names = data.columns
    for name in col_names:
        if name.endswith('_p'):
            match_cols.append(name)
            new_cols.append(name.strip())

    ext_data = pd.DataFrame(data[match_cols])
    ext_data.columns = new_cols
    
    grand_total = ext_data['tot_p'].sum(axis = 0)
    country_total = ext_data.sum(axis = 0)
    percentage = country_total/grand_total * 100
    
    birth_country = pd.DataFrame(country_total, columns = ['country_total'])
    birth_country['percentage'] = percentage
    
    drop_columns = ['hong_kong_sar_china_p', 'born_elsewhere_p', 'tot_p', 'os_visitors_p', 'country_birth_not_stated_p', 'australia_p', 'new_zealand_p', 'united_states_america_p', 'united_kingdom_ci_im_p', 'fiji_p', 'south_africa_p']
    
    birth_country.loc['china_excl_sars_taiwan_p'] += birth_country.loc['hong_kong_sar_china_p']
    birth_country = birth_country.T.drop(drop_columns, axis = 1)
    birth_country = birth_country.rename({'china_excl_sars_taiwan_p' : 'china_p', 'sri_lanka_p' : 'srilanka_p'}, axis = 1)
    
    country_names = []
    for item in birth_country.columns:
        item = item.split('_')
        country_names.append(item[0].capitalize())

    birth_country.columns = country_names
    birth_country = birth_country.rename({'Srilanka' : 'Sri Lanka'}, axis = 1).T
    birth_country = birth_country.sort_values(by = ['country_total'], ascending = False)

    birthcountry_db = db_connect('birthcountry')
    for i in range(len(birth_country)):
        birthcountry_db.save({'country': birth_country.index[i], 'values' : (birth_country.country_total.values[i], birth_country.percentage.values[i])})

In [84]:
def lang_spoken_at_home(file_path):
    """
    extract top N languages other than English spoken at home
    params: path to census data download from AURIN - 'lang_at_home.csv';
            path to langCode.json file;
            number of languages other than English to extract
    return: names of top N languages other than English spoken at home, total population count, percentage of population count to total SOL population, 
            percentage of population count to total population, and percentage of SOL population to total population
    return type: numpy arrays
    frontend: bar chart/pie chart (colour matching for most tweeted languages/counrty of birth/language spoken at home)
    """
    
    data = pd.read_csv(file_path)

    match_cols = []
    new_cols = []
    col_names = data.columns
    for name in col_names:
        if name.endswith('_P'):
            match_cols.append(name)   
            new_cols.append(name.strip())

    ext_data = pd.DataFrame(data[match_cols])
    ext_data.columns = new_cols

    SOL_tot = ext_data['SOL_Tot_P'].sum(axis = 0)
    tot = ext_data['Total_P'].sum(axis = 0)
    SOL_perc = SOL_tot/tot * 100

    drop_columns = ['SOL_Other_P', 'SOL_Samoan_P', 'SOL_Assyrian_P','SOL_Iran_Lan_Tot_P', 
                    'SOL_Irani_Lan_Othr_P', 'SOL_Se_As_A_L_Othr_P', 'SOL_Aus_Indig_Lang_P', 
                    'SOL_In_Ar_Lang_Othr_P', 'SOL_In_Ar_Lang_Tot_P', 'SOL_Se_As_A_L_Tot_P', 
                    'Language_spoken_home_ns_P', 'SOL_Tot_P', 'Total_P', 
                    'SOL_Chin_lang_Mand_P', 'SOL_Chin_lang_Other_P', 'SOL_Chin_lang_Cant_P']
    
    ext_data = ext_data.drop(drop_columns, axis = 1)
    lang_tot = ext_data.sum(axis = 0)
    
    columns = []
    for index in lang_tot.index:
        idx = index.split('_')
        if 'Se_As' in index or 'In_Ar' in index or 'Ir_Lang' in index:
            columns.append(idx[-2])
        elif 'Ir_La' in index:
            columns.append(idx[3])
        else:
            columns.append(idx[1])

    lang_tot.index = columns
    lang_data = pd.DataFrame(lang_tot, columns = ['number']).T
    lang_data = lang_data.rename({'Pe' : 'Persian'}, axis = 1).T
    lang_data['percentage_SOL'] = lang_data['number']/SOL_tot * 100
    lang_data['percentage_Total'] = lang_data['number']/tot * 100

    langCode = read_langCode()
    langdict = {k:v for v in langCode.values() for k in lang_data.index if k in v}

    idx = []
    for i in lang_data.index:
        name = langdict[i]
        idx.append(name)

    lang_data.index = idx
    lang_data = lang_data.sort_values(by = ['number'], ascending = False)
    
    homelang_db = db_connect('homelang')
    for i in range(len(lang_data)):
        array = np.array([lang_data.number.values[i], lang_data.percentage_SOL.values[i], lang_data.percentage_Total.values[i]])
        values = array.tolist()
        homelang_db.save({'country': lang_data.index[i], 'values' : values})